In [ ]:
from Bio import SeqIO
import os
import pandas as pd

## Uniprot header format

UniProtKB

    >db|UniqueIdentifier|EntryName ProteinName OS=OrganismName OX=OrganismIdentifier [GN=GeneName ]PE=ProteinExistence SV=SequenceVersion

Where:

- *db* is 'sp' for UniProtKB/Swiss-Prot and 'tr' for UniProtKB/TrEMBL.
- *UniqueIdentifier* is the primary accession number of the UniProtKB entry.
- *EntryName* is the entry name of the UniProtKB entry.
- *ProteinName* is the recommended name of the UniProtKB entry as annotated in the RecName field. For UniProtKB/TrEMBL entries without a RecName field, the SubName field is used. In case of multiple SubNames, the first one is used. The 'precursor' attribute is excluded, 'Fragment' is included with the name if applicable.
- *OrganismName* is the scientific name of the organism of the UniProtKB entry.
- *OrganismIdentifier* is the unique identifier of the source organism, assigned by the NCBI.
- *GeneName* is the first gene name of the UniProtKB entry. If there is no gene name, OrderedLocusName or ORFname, the GN field is not listed.
- *ProteinExistence* is the numerical value describing the evidence for the existence of the protein.
- *SequenceVersion* is the version number of the sequence.

Examples

    >sp|P02489|CRYAA_HUMAN Alpha-crystallin A chain OS=Homo sapiens OX=9606 GN=CRYAA PE=1 SV=2
    >sp|P04224|HA22_MOUSE H-2 class II histocompatibility antigen, E-K alpha chain OS=Mus musculus OX=10090 PE=1 SV=1
    >tr|Q8N2H2|Q8N2H2_HUMAN cDNA FLJ90785 fis, clone THYRO1001457, moderately similar to H.sapiens protein kinase C mu OS=Homo sapiens OX=9606 PE=2 SV=1



# Saul 2023
fasta file downloaded from [Saul et al (2023)](https://doi.org/10.5061/dryad.xsj3tx9mj)

    >Nepenthes gracilis||scaffold1||1982839||1989396||Nepgr013152.mRNA1||1||CDS||3477145685||1||frame0
    MGFIHWNYSAIVYNITMISFNMLETARINGVKRFFYAPSASIYPEFRQWKTTSRGRREKAPAAFCRKAIASTDRFEMWGDRKQTRSLTFIDECVEDAPISDLGKIGSSHVDDIFSCYLRHLAFSGPYKIISHSENRLFWWIAVRFPNTFLPENIEPVIQRVRLYELVNIGGDEMVGMNEVADIVLSFENKKLHIHNLSWAPMMKLKVTFELF

Nepgr013152 is the identifier that has to bee matched with the metadata provided by the authors.

## Load data

In [ ]:
# define directories
# get the parent of the current directory
PROJ_DIR = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
DATA_DIR = os.path.join(PROJ_DIR, "data", "genomes", "nepenthes_SAUL2023")
DATA_DIR

## Explore data

In [ ]:
infile = os.path.join(DATA_DIR, "61566-CDS-prot.fasta")
metadata_file = os.path.join(DATA_DIR, "Nepenthes_gracilis.annotation.tsv")

# parse infile
records = list(SeqIO.parse(infile, "fasta"))

# get total number of reads
total_reads = len(records)
total_reads

In [ ]:
# look at structure of records
record = records[1]
record

In [ ]:
# extract information from the record description
organism, scaffold, start, end, gene_id, strand, region, temp, temp1, frame = (
    record.description.split("||")
)
gene_id = gene_id.split(".")[0]
print(organism)
print(gene_id)

## Explore metadata

In [ ]:
pd.set_option("display.max_columns", 8)
metadata = pd.read_csv(metadata_file, sep="\t")
metadata.head()

relevant columns are
gene_id, sprot_best, sprot_recname,  start, end.\
orthogroup, go_ids could be of interest later on.


UniProtKB

    >db|UniqueIdentifier|EntryName ProteinName OS=OrganismName OX=OrganismIdentifier [GN=GeneName ]PE=ProteinExistence SV=SequenceVersion

new header:

        >tr|gene_id|sprot_best sprot_recname OS=organism OX=150966 GN=gene_id PE=3 SV=1


In [ ]:
# get the annotation information
taxid = 150966
gene_id = "Nepgr000005"
# find information in metadata where "gene_id" contains the gene_id
gene_info = metadata[metadata["gene_id"].str.contains(gene_id)]
entry_name = gene_info["sprot_best"].values[0]
protein_name = gene_info["sprot_recname"].values[0]

# Print the record with new header
print(
    f">tr|{gene_id}|{entry_name} {protein_name} OS={organism} OX={taxid} GN={gene_id} PE=3 SV=1"
)
print(f"{record.seq}\n")

## Test

In [ ]:
for i, record in enumerate(SeqIO.parse(infile, "fasta")):
    if i == 2:
        break

    # extract information from the record description
    organism, scaffold, start, end, gene_id, strand, region, temp, temp1, frame = (
        record.description.split("||")
    )
    gene_id = gene_id.split(".")[0]

    # find information in metadata where "gene_id" contains the gene_id
    gene_info = metadata[metadata["gene_id"].str.contains(gene_id)]
    entry_name = gene_info["sprot_best"].values[0]
    protein_name = gene_info["sprot_recname"].values[0]

    # Print the record with new header
    print(
        f">tr|{gene_id}|{entry_name} {protein_name} OS={organism} OX={taxid} GN={gene_id} PE=3 SV=1"
    )
    print(f"{record.seq}\n")

## Annotate fasta file

In [ ]:
outfile = os.path.join(DATA_DIR, "61566-CDS-prot-annotated.fasta")
taxid = 150966

with open(outfile, "w") as of:
    for record in SeqIO.parse(infile, "fasta"):

        # extract information from the record description
        organism, scaffold, start, end, gene_id, strand, region, temp, temp1, frame = (
            record.description.split("||")
        )
        gene_id = gene_id.split(".")[0]

        # find information in metadata where "gene_id" contains the gene_id
        gene_info = metadata[metadata["gene_id"].str.contains(gene_id)]
        entry_name = gene_info["sprot_best"].values[0]
        protein_name = gene_info["sprot_recname"].values[0]

        # Print the record with new header
        of.write(
            f">tr|{gene_id}|{entry_name} {protein_name} OS={organism} OX={taxid} GN={gene_id} PE=3 SV=1\n"
        )
        of.write(f"{record.seq}\n")